## Устанвливаем всякое нужное

In [20]:
from nltk.tokenize import word_tokenize
import nltk
from string import punctuation
import pymorphy2
from nltk.stem.snowball import RussianStemmer
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')

#исходные текстовые данные
string = 'Криптовалюты - это наше будущее! Очень хотелось бы, чтобы эта технология скорее получила большее признание. И мне хотелось бы на этом заработать)'

[nltk_data] Downloading package punkt to /Users/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexander/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Читаем файлик

In [22]:
df = pd.read_csv('res.csv', sep=";")
df.head()

,Unnamed: 0,Transcription,Emotion,Sentiment,Emotions
0,0,Давай ты успокоишься для начала. Ты можешь не ...,гнев,негативный сентимент,0
1,1,"Подожди, подожди, успокойся пожалуйста. Подожд...",гнев,негативный сентимент,0
2,2,"Кристин, привет. Ты слышала там самолёт упал в...",страх,негативный сентимент,5
3,3,"Кристин, привет .. слушай .. Я боюсь лететь, т...",страх,негативный сентимент,5
4,4,"Почему? Подожди... Ну подожди, ну как же, но м...",грусть,негативный сентимент,1


## Функция со всеми шагами обработки.  Константы выносим в аргументы, чтобы не пересчитывать

In [31]:
def preprocessing(string, stopwords, morph, punctuations) -> str:
    tokens = word_tokenize(string)

    tokens_without_punct = [i for i in tokens if i not in punctuations]
    low_tokens  = [t.lower() for t in tokens_without_punct]
    words_without_stop = [w for w in low_tokens if w not in stopwords]
    lemms = [morph.parse(i)[0].normal_form for i in words_without_stop]
    result_string = ""
    for i in lemms:
        result_string+=(i+" ")
    return result_string

## Делаем константные списки. Удаляем из пунктуации скобочку (для улыбки)

In [40]:
punctuations = list(punctuation)
punctuations.remove(")")

In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

## Результат работы функции

In [45]:
preprocessing(string, stopwords, morph, punctuations)

'криптовалюта это наш будущее очень хотеться этот технология скорее получить больший признание хотеться заработать ) '

## Делаем список подготовленных текстов и записываем его в датафрейм

In [42]:
preprocessed_text = [preprocessing(x, stopwords, morph, punctuations) for x in df.Transcription]


In [43]:
df["Preprocessed_text"]=preprocessed_text
df

,Unnamed: 0,Transcription,Emotion,Sentiment,Emotions,Preprocessed_text
0,0,Давай ты успокоишься для начала. Ты можешь не ...,гнев,негативный сентимент,0,давать успокоиться начало мочь вестись ерунда ...
1,1,"Подожди, подожди, успокойся пожалуйста. Подожд...",гнев,негативный сентимент,0,подождать подождать успокоиться пожалуйста под...
2,2,"Кристин, привет. Ты слышала там самолёт упал в...",страх,негативный сентимент,5,кристин привет слышать самолёт упасть зеландия...
3,3,"Кристин, привет .. слушай .. Я боюсь лететь, т...",страх,негативный сентимент,5,кристин привет .. слушать .. бояться лететь ра...
4,4,"Почему? Подожди... Ну подожди, ну как же, но м...",грусть,негативный сентимент,1,почему подождать ... подождать давно планирова...
...,...,...,...,...,...,...
530,530,Я тут готовила медовик. Чтобы приготовить медо...,нейтраль,нейтральный сентимент,2,готовить медовик приготовить медовик очень вре...
531,531,"Значит, правила игры в дурака. Обычного. 36 ка...",нейтраль,нейтральный сентимент,2,значит правило игра дурак обычный 36 карта кол...
532,532,Так. В Третьяковской галерее продается такая и...,нейтраль,нейтральный сентимент,2,третьяковский галерея продаваться такой игра п...
533,533,"я из города Челябинска. Ничего не умел, ничего...",нейтраль,нейтральный сентимент,2,город челябинск уметь хотеть учиться банкир фи...


## Сохраняемся

In [46]:
df.to_csv('res2.csv', sep=";", index=False)